In [118]:
import pandas as pd
from ortools.linear_solver import pywraplp
import numpy as np

dataset = pd.read_csv("../Downloads/truc/faces.utf8.csv")
dataset = dataset.values
briefs = pd.read_csv("../Downloads/truc/briefs.utf8.csv")
briefs = briefs.values
result = []

def get_week(week, data):
    array = []
    for i in range(0, len(data)):
        if data[i][0] == week:
            array.append(data[i])
    return (array)

dict_col = {}
dict_col["25_49csp"] = 12
dict_col["cspp"] = 15
dict_col["urb"] = 18
dict_col["all"] = 14
dict_col["f25_49rda"] = 17
dict_col["f25_49csp"] = 16
dict_col["15_24"] = 10
dict_col["25_49"] = 11
dict_col["25_49urb"] = 13

def get_max_week():
    return (briefs[len(briefs)-1][0])

def get_coeff(data, contactkey):
    

def solve_week(week_nb, result):
    brief = [[]]
    full_week = get_week(week_nb, briefs)
    data = dataset.tolist()
    for k in range (0, len(full_week)):
        brief[0] = full_week[k]
        contactkey = dict_col[brief[i][2]]
        solver = pywraplp.Solver('SolveStigler', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

        faces = [[]] * len(data)
        objective = solver.Objective()
        for i in range(0, len(data)):
            faces[i] = solver.NumVar(0, 1, data[i][0])
            objective.SetCoefficient(faces[i], get_coeff(data[i], contactkey))
        objective.SetMinimization()

        constraints = [0] * len(brief)
        for i in range(0, len(brief)):
            constraints[i] = solver.Constraint(brief[i][3], solver.infinity())
            for j in range(0, len(data)):
                constraints[i].SetCoefficient(faces[j], data[j][contactkey])
                

        status = solver.Solve()
        ref = []
        if status == solver.OPTIMAL:
            nb_faces = 0
            re = []
            indexes = []
            for i in range (0, len(data)):
                if (faces[i].solution_value() > 0):
                    nb_faces += 1
                    indexes.append(i)
                    re.append(data[i][0])
            result.append(re)
            for i in reversed(indexes):
                data.pop(i)
        else:
            if status != solver.FEASIBLE:
                result.append(ref)
    return (result)

for i in range(1, get_max_week()):
    result = solve_week(i, result)

In [116]:
def make_key(week, order):
    return '"(week-' + str(week) + ", order-" + str(order) + ')"'
def make_value(sol):
    ret = "["
    l = len(sol)
    for i in range(0, l):
        ret += '"'+str(sol[i])+'"'
        if i != l - 1:
            ret +=","
    ret += "]"
    return ret
def to_submit(sols):
    ret = "{"
    l = len(sols)
   # if l != len(briefs):
    #    return "Error. The number of briefs and number of results are not matching."
    for i in range(0, l):
        if len(sols[i]) != 0:
            ret += make_key(briefs[i][0], briefs[i][1]) + ":" + make_value(sols[i])
            if i != l -1:
                ret += ","
    if ret[-1] == ',':
        ret = ret[:-1]
    ret += "}"
    return ret

In [113]:
import json
s = to_submit(result)
print(s)
test = json.JSONDecoder()
test.decode(s)

{"(week-1, order-1)":["69382.00041.01.01.01","75110.10408.03.01.01","75112.00155.09.01.02","75112.00295.04.01.01","75112.00772.01.02.01","75112.00772.01.02.02","75113.00201.02.01.01","75113.00201.02.01.02","75113.00201.02.01.03","75113.00201.02.01.04","75113.00201.02.02.01","75113.00201.02.02.02","75113.00201.02.02.03","75113.00201.02.02.04","75113.00258.01.01.01","75113.00258.01.01.02","75113.00258.01.01.03","75113.00258.01.01.04","75113.00838.01.01.02","75113.00847.01.02.01","75113.00847.01.02.02","75113.01349.10.01.01","75113.01349.10.01.02","75113.01349.10.01.03","75113.01349.10.01.04","75113.01349.11.01.01","75113.01349.11.01.02","75113.01349.11.01.03","75113.01349.11.01.04","75113.01349.12.01.01","75113.01349.12.01.02","75113.01349.12.01.03","75113.01349.12.01.04","75113.01349.13.01.01","75113.01349.13.01.02","75113.01349.13.01.03","75113.01349.13.01.04","75114.00015.02.01.01","75114.00279.02.02.01","75114.00279.02.02.02","75114.00279.02.02.03","75114.00279.02.02.04","75114.00280

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [117]:
f = open("answer.json", "w")
f.write(s)
f.close()